In [ ]:
%reset -f

In [ ]:
import os
os.chdir(r'/home')
import numpy as np
import scipy
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# GPU
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Functions
def derivee(y,x):
  return torch.autograd.grad(y,x,torch.ones_like(y),retain_graph=True,create_graph=True)[0]

def som(s):
  return torch.sum(s,dim=-1,keepdim=True)

namedata = 'Struct_1_512'
Data = scipy.io.loadmat(namedata+'.mat')
pix = torch.tensor(Data['pixels'], dtype = torch.float32).to(device)

In [ ]:
# NN module
class MLP_gelu(nn.Module):
    def __init__(self,insize,outsize,width,layers):
        super().__init__()
        self.input_layer = nn.Linear(insize,width)
        self.hidden_layers = nn.ModuleList([])
        self.output_layer = nn.Linear(width,outsize)
        for i in range(layers):
          self.linear_layer = nn.Linear(width,width)
          self.hidden_layers.append(self.linear_layer)
        self.layers = layers

    def forward(self, x):
        layer_out = F.gelu(self.input_layer(x))
        for i in range(self.layers):
            layer_out = F.gelu(self.hidden_layers[i](layer_out)) # + layer_out
        layer_out = self.output_layer(layer_out)
        return layer_out

# Neural network in PINN
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.NN_trunk = MLP_gelu(insize=4,outsize=2,width=64,layers=3)

    def forward(self, x, y, e):
        # Normalisation
        scaler = torch.max(torch.abs(e))
        e = e/scaler
        # NN modulus
        U_tilde = self.NN_trunk(torch.cat([torch.sin(x * 2*torch.pi),
                                     torch.cos(x * 2*torch.pi),
                                     torch.sin(y * 2*torch.pi),
                                     torch.cos(y * 2*torch.pi)], dim=-1))
        ux_tilde,uy_tilde = U_tilde[...,0:1],U_tilde[...,1:2]
        # Periodic Boundary condition
        ux = (ux_tilde + e[0]*x + e[2]*y)*scaler
        uy = (uy_tilde + e[2]*x + e[1]*y)*scaler
        return ux, uy

class Homogenisation():
    def __init__(self, struct, epsilon_bar):
        super(Homogenisation, self).__init__()

        # Configuration
        self.struct = struct
        # Average Strain
        self.epsilon_bar = epsilon_bar
        # Constitutive Model: Elastic Modulus and Poisson's ratio
        self.E, self.nu = 10.0, 0.3
        # Solution (Displacement)
        self.U = PINN().to(device)
        # Coordinates
        Ndim = len(struct)
        X, Y = torch.meshgrid(torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                              torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                              indexing='ij')
        L_pde = (self.struct.reshape(-1) == 1).to(device)
        self.x_pde = X.reshape(-1)[:,None].to(device)[L_pde]
        self.y_pde = Y.reshape(-1)[:,None].to(device)[L_pde]
        self.x_pde.requires_grad = True
        self.y_pde.requires_grad = True
        # Iteration
        self.iter = 0
        self.scaler = torch.max(torch.abs(epsilon_bar))

    def Phi(self): # strain Energy (linear Elasticity)
        # Displacement
        ux, uy = self.U(self.x_pde, self.y_pde, self.epsilon_bar)
        # Strain
        epsilon_xx = derivee(ux,self.x_pde)
        epsilon_yy = derivee(uy,self.y_pde)
        epsilon_xy = (derivee(ux,self.y_pde) + derivee(uy,self.x_pde))/2.0
        # Constitutive
        pref  = self.E / ((1.0 + self.nu) * (1.0 - 2.0 * self.nu))
        G   = self.E / (2.0 * (1.0 + self.nu))
        # Stress
        sigma_xx = pref * ((1.0 - self.nu) * epsilon_xx + self.nu * epsilon_yy)
        sigma_yy = pref * (self.nu * epsilon_xx + (1.0 - self.nu) * epsilon_yy)
        sigma_xy = 2.0 * G * epsilon_xy
        # Strain Energy
        phi = 0.5*(epsilon_xx*sigma_xx + epsilon_yy*sigma_yy + 2.0*epsilon_xy*sigma_xy)/self.scaler**2
        return phi.mean()

    def Solution(self): # Calculate stress field
        # Displacement
        ux, uy = self.U(self.x_pde, self.y_pde, self.epsilon_bar)
        # Strain
        epsilon_xx = derivee(ux,self.x_pde)
        epsilon_yy = derivee(uy,self.y_pde)
        epsilon_xy = (derivee(ux,self.y_pde) + derivee(uy,self.x_pde))/2.0
        # Constitutive
        pref  = self.E / ((1.0 + self.nu) * (1.0 - 2.0 * self.nu))
        G   = self.E / (2.0 * (1.0 + self.nu))
        # Stress
        sigma_xx = pref * ((1.0 - self.nu) * epsilon_xx + self.nu * epsilon_yy)
        sigma_yy = pref * (self.nu * epsilon_xx + (1.0 - self.nu) * epsilon_yy)
        sigma_xy = 2.0 * G * epsilon_xy
        # Assamble
        U = torch.cat((ux,uy),dim=-1)
        Epsilon = torch.cat((sigma_xx,sigma_yy,sigma_xy),dim=-1)
        Sigma = torch.cat((sigma_xx,sigma_yy,sigma_xy),dim=-1)
        return U, Epsilon, Sigma

    def closure(self): # closure function for L-BSGF optimisor
        self.iter += 1
        optimizer.zero_grad()
        Loss = self.Phi()
        Loss.backward()
        if (self.iter+1)%100==1:
            print(f'Epoch:{self.iter}, PDE Loss:{Loss:.3e}')
        return Loss

In [ ]:
Macro_Epsilon = torch.tensor([0.0,1.0,0.0]).to(device)
model = Homogenisation(pix,Macro_Epsilon)

lr = 1.0
optimizer = torch.optim.LBFGS(model.U.parameters(),lr=lr,
                              max_iter=1000,
                              max_eval=None,
                              tolerance_grad=0,
                              tolerance_change=0,
                              history_size=100)

optimizer.step(model.closure)

In [ ]:
U, Epsilon, Sigma = model.Solution()

x = model.x_pde.detach().cpu().numpy()
y = model.y_pde.detach().cpu().numpy()
ux, uy = U[:, 0].detach().cpu().numpy(), U[:, 1].detach().cpu().numpy()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.5))
ax1.set_title('ux'); ax1.axis('off')
ax2.set_title('uy'); ax2.axis('off')
plt.colorbar(ax1.scatter(x, y, c=ux, cmap='jet', s=0.12), ax=ax1)
plt.colorbar(ax2.scatter(x, y, c=uy, cmap='jet', s=0.12), ax=ax2)
plt.tight_layout()
plt.show()